## Cohere Tutorial

In [4]:
import cohere
import numpy as np

cohere_key = "COHERE API KEY"
co = cohere.Client(cohere_key)

docs = ["The capital of France is Paris",
        "PyTorch is a machine learning framework based on the Torch library.",
        "The average cat lifespan is between 13-17 years"]

doc_emb = co.embed(texts=docs, input_type="search_document", model="embed-multilingual-v3.0").embeddings
doc_emb = np.asarray(doc_emb)

query = "What is Pytorch"
query_emb = co.embed(texts=[query], input_type="search_query", model="embed-multilingual-v3.0").embeddings
query_emb = np.asarray(query_emb)

scores = np.dot(query_emb, doc_emb.T)[0] # dot product between query embedding and document embedding

#Find the highest scores
max_idx = np.argsort(-scores)

print(f"Query: {query}")
for idx in max_idx:
        print(f"Score: {scores[idx]:.2f}")
        print(docs[idx])
        print("--------")

Query: What is Pytorch
Score: 0.63
PyTorch is a machine learning framework based on the Torch library.
--------
Score: 0.24
The capital of France is Paris
--------
Score: 0.20
The average cat lifespan is between 13-17 years
--------


## Preprocessing

In [5]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

def clean(text:str):
    not_used = re.compile('[^ .?!/@$%~|0-9|ㄱ-ㅣ가-힣]+')
    dup_space = re.compile('[ \t]+')  # white space duplicate
    dup_stop = re.compile('[\.]+')  # full stop duplicate
    cleaned = not_used.sub('', text.strip()) 
    cleaned = dup_space.sub(' ', cleaned)
    cleaned = dup_stop.sub('.', cleaned) 
    return cleaned

df = pd.read_csv('./BalancedNewsCorpusShuffled/BalancedNewsCorpus_train.csv')
df['News'] = df['News'].apply(lambda text: text.replace('<p>', '\n').replace('</p>', '\n'))
df['News'] = df['News'].apply(clean)
df

,filename,date,NewsPaper,Topic,News
0,NLRW1900000141,20170324,부산일보,스포츠,야구 종가 마침내 정상에 서다 야구 종가 미국이 푸에르토리코를 누르고 2017 월드...
1,NPRW1900000003,20110209,한국경제신문사,정치,외통위 27명중 15명 추가협상안만 처리 국회 외교통상통일위원회 소속 의원 27명 ...
2,NLRW1900000144,20100406,영남일보,사회,한나라 후보 희망연대 당원 구함 공천변수 작용 주목 오늘까지 추가 모집 오는 6월 ...
3,NLRW1900000064,20100804,광주매일신문,스포츠,모처럼 살아난 포 7타점 합작 12 3 4강 진입을 놓고 혈전을 벌이고 있는 가 와...
4,NLRW1900000070,20160615,광주매일신문,문화,문화전당서 동방의 등불 만나다 일찍이 아시아의 황금 시기에 빛나던 등불의 하나였던 ...
...,...,...,...,...,...
8995,NWRW1900000006,20141114,조선일보사,IT/과학,내가 구매한 영화 출근길의 오빠도 방에 있는 엄마도 보네? 스마트폰 가족 공유 콘텐...
8996,NIRW1900000022,20101217,노컷뉴스,연예,꽃보다 예쁜 터치 선웅의 여장 이게 바로 안산 4 수퍼 루키 터치의 멤버 선웅이 여...
8997,NLRW1900000092,20180131,국제신문,사회,어머니 빨리 쾌차하세요밀양참사 후 더 깊어진 고부애 같은 병실 입원해 극진히 수발 ...
8998,NLRW1900000103,20090206,대전일보,IT/과학,엑스포공원 드라마 타운 성공하려면 대전 엑스포 과학공원 내 조성될 것으로 기대를 모...


## Milvus Connect & Create Collection

In [11]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

DATASET = 'BalancedNewsCorpus'  
MODEL = 'embed-multilingual-v3.0' # Cohere/embed-multilingual-v3.0
COLLECTION_NAME = 'COHERE_IP'  # Collection name
EMBEDDING_FIELD_NAME ='embedding'
INDEX_TYPE = "IVF_FLAT"

# 인덱스 설정
INDEX_PARAMS = {
    'metric_type': 'IP',     # "L2", "IP", "COSINE" 
    'index_type': INDEX_TYPE,    # "IVF_FLAT","IVF_SQ8", "IVF_PQ", "HNSW", "ANNOY"
    'params': {"nlist": 128},
}

def milvus_connect(collection_name):
    # Milvus 설정
    DIMENSION = 1024  # Cohere/embed-multilingual-v3.0의 dimension
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530'     

    # Milvus 연결
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if connections:
        print("Milvus connected")
    else:
        exit()

    # 컬렉션 존재 여부 확인 및 생성
    if utility.has_collection(collection_name):
        collection = Collection(collection_name)
        print(f"Collection '{collection_name}' loaded.")
    else:
        print(f"Collection '{collection_name}' does not exist.")
        print(f"Create '{collection_name}' Collection.")
        
        field_args = [
            FieldSchema(name='file_id', dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name='filename', dtype=DataType.VARCHAR, max_length=20),
            FieldSchema(name='date', dtype=DataType.INT64, max_length=10),
            FieldSchema(name='NewsPaper', dtype=DataType.VARCHAR, max_length=100),
            FieldSchema(name='Topic', dtype=DataType.VARCHAR, max_length=100),
            FieldSchema(name='News', dtype=DataType.VARCHAR, max_length=30000),
            FieldSchema(name=EMBEDDING_FIELD_NAME, dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
        ]
        
        # 스키마 정의
        schema = CollectionSchema(fields=field_args)
        
        # 컬렉션 생성
        collection = Collection(name=collection_name, schema=schema)
        
        # 인덱스 생성
        collection.create_index(field_name=EMBEDDING_FIELD_NAME, index_params=INDEX_PARAMS)
        collection.load()
        print(f"Collection '{collection_name}' created and loaded.")
    return collection

In [12]:
collection = milvus_connect(COLLECTION_NAME)

Milvus connected
Collection 'COHERE_IP' does not exist.
Create 'COHERE_IP' Collection.
Collection 'COHERE_IP' created and loaded.


In [10]:
# # Collection 제거용
# from pymilvus import connections, utility
# MILVUS_HOST = '127.0.0.1'
# MILVUS_PORT = '19530' 
# COLLECTION_NAME = 'COHERE_IP'
# connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
# if utility.has_collection(COLLECTION_NAME):
#     utility.drop_collection(COLLECTION_NAME)

## Embedding with Cohere embedding model

In [13]:
import cohere
import numpy as np

cohere_key = "COHERE API KEY"
co = cohere.Client(cohere_key)

# 임베딩할 텍스트 로드
texts = df['News'].tolist()
doc_emb = co.embed(texts=texts, input_type="search_document", model="embed-multilingual-v3.0").embeddings
embeddings = np.asarray(doc_emb)
embeddings.shape

(9000, 1024)

## Dataset & Insertion

In [14]:
column2field = {
    'filename':'filename',
    'date':'date',
    'NewsPaper':'NewsPaper',
    'Topic':'Topic',
    'News':'News',
}

field2column = {v:k for k,v in column2field.items()}

class CustomDataset(Dataset):
    def __init__(self, df, embeddings, field2column, embedding_field):
        self.df = df
        self.embeddings = embeddings
        self.field2column = field2column
        self.embedding_field = embedding_field
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        items = {field: self.df.iloc[idx][column] for field, column in self.field2column.items()}
        items[self.embedding_field] = self.embeddings[idx].tolist()
        return items
    
dataset = CustomDataset(df, embeddings, field2column, EMBEDDING_FIELD_NAME)

In [15]:
# insert 수행하기
for data in tqdm(iter(dataset), total=len(dataset)):
    collection.insert(data)
    
collection.flush()
collection.num_entities # 9000, 삽입된 엔티티의 수를 나타낸다.

100%|██████████| 9000/9000 [03:12<00:00, 46.80it/s]


9000

## Vector Search

In [16]:
test_df = pd.read_csv('./BalancedNewsCorpusShuffled/BalancedNewsCorpus_test.csv')
test_df['News'] = test_df['News'].apply(lambda text: text.replace('<p>', '\n').replace('</p>', '\n'))
test_df['News'] = test_df['News'].apply(clean)

test_idx = 123
query = test_df.iloc[test_idx]['News']
query

'카울리 클라우드 총괄 클라우드시장 금융서 폭발성장 물꼬만 터진다면 한국 클라우드 시장은 폭발적으로 성장할 것으로 예상합니다. 에서 글로벌 클라우드 시장을 총괄하고 있는 스티브 카울리 총괄 대표는 최근 매일경제신문과 인터뷰를 갖고 한국은 시장 규모에 비해 클라우드 도입 정도가 매우 낮다며 이같이 말했다. 카울리 대표는 예를 들어 금융산업 같은 경우 보안 이슈와 규제 문제 때문에 클라우드 도입을 주저하고 있는 게 사실이라면서도 처음 클라우드를 도입하는 은행은 여러 가지 이점을 선점할 수 있기 때문에 빠른 확산이 가능하다고 전망했다. 실제로 호주 벤디고은행은 톱4에 들지 못하는 하위 은행이었지만 클라우드블루믹스를 도입한 뒤 100년 넘은 은행이라는 전통적 이미지를 벗고 혁신 선도 은행으로 이미지를 새롭게 정립했다고 그는 설명했다. 한국 시장에 대한 낙관적 전망으로 은 지난해 퍼블릭 클라우드 데이터센터를 건설하기도 했다. 카울리 대표는 클라우드 서비스가 가격경쟁으로 가고 있지만 가격보다 가치라는 변수가 더 중요하다면서 한국에서 다수 잠재 고객사들을 만나고 난 뒤 새로운 클라우드 서비스를 도입하는 것은 어떤 가치를 제공해 주느냐의 싸움이라는 것을 더 확신하게 됐다고 강조했다. 특히 처럼 이미 한국에 비즈니스 고객들이 많았던 회사들이 클라우드 서비스를 제공한다면 수요자 니즈를 더 잘 파악할 수 있을 것이라고 했다. 카울리 대표는 은 인공지능 왓슨 블록체인 사물인터넷 데이터 애널리틱스 등 다양한 솔루션도 갖추고 있다고 부연했다. 카울리 대표는 앞으로 정부 시스템도 클라우드로 대체될 것이라는 흥미로운 전망도 했다. 그는 시민들이 더 많이 참여하는 정치 형태를 선호하면서 행정수요 처리 방식을 혁신할 필요성이 증가하고 있다며 시민과 정부 사이 관계를 개선하는 데 클라우드 서비스가 도움이 된다는 평가도 나오고 있다고 말했다. 카울리 대표는 세계적으로 보면 공공부문은 클라우드 사업의 주요한 시장이라며 해당 분야에 대한 연구를 강화하고 있다고 말했다. 은 최근 19분기 연속으로 전

In [18]:
query_embedding = co.embed(texts=[query], input_type="search_query", model="embed-multilingual-v3.0").embeddings
query_embedding = np.asarray(query_emb)
# query_embedding.shape # (1, 1024)
query_embedding = query_embedding.reshape(-1)
# query_embedding.shape # (1024, )

result = collection.search(
    data = [query_embedding], # 앞서 생성한 동일한 크기의 임베딩 벡터 
    anns_field = EMBEDDING_FIELD_NAME, # 어떤 임베딩 필드를 기준으로 검색할 지 선언
    param = INDEX_PARAMS, # 생성했었을 때의 동일한 index parameter
    limit = 5, # top K를 뜻하며 검색해서 보고 싶은 개수
    output_fields = ['filename','date', 'NewsPaper', 'Topic', 'News'] # 보고 싶은 필드 명시
)

In [16]:
# COHERE (nlist : 1, metric_type : COSINE)
result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451154865982061039,
  'distance': 0.3620307147502899,
  'entity': {'Topic': 'IT/과학',
   'News': '빨강초록파랑 빛의 연금술사 색안경을 쓰면 세상이 왜곡돼 보인다. 이 속성을 그래픽 디자인으로 승화했다. 빛의 삼원색인 빨강 초록 파랑이 마구 섞인 요란하고 어지러운 그림. 그 속엔 세 개의 서로 다른 그림이 숨겨져 있다. 플라스틱 필터를 눈앞에 대는 순간 맨눈으로 볼 때와 전혀 딴판인 그림이 등장한다. 이 독창적 작품을 만든 팀은 카르노브스키. 이탈리아인 프란체스코 루지41와 콜롬비아인 실비아 퀸타닐라39 부부로 이뤄진 그룹이다. 카르노브스키는 부부가 좋아하는 소설 속 주인공 이름에서 따왔다. 2012년 영국 유명 건축디자인 잡지 월페이퍼가 주는 월페이퍼 디자인 어워드에서 수상하며 주목받기 시작했다. 그림들에 세 가지 색 조명을 비추거나 필터를 대는 순간 마법이 펼쳐진다. 작품 랜드스케이프 .2를 빨간색 필터를 통해 보면 들판 위 큰 나무와 집이 있는 풍경이 나오고 초록 필터를 통해 보면 우거진 풀숲이 파란색 필터를 통해 보면 해안가 도시와 바다를 항해하는 배의 모습이 보인다. 필터와 같은 색의 그림은 보이지 않고 보색의 그림이 선명하게 보이면서 숨겨진 풍경이 드러나는 것이다. 세 가지 그림이 서로 밀접하게 연관된 작품도 있다. 서 있는 사람을 소재로 한 베살리오 .2는 빨간색을 비추면 뼈 초록색을 비추면 근육 파란색을 비추면 혈관만 보인다. 빨강 초록 파랑 순으로 심연에 다가선다고 보면 된다. 빨간색 필터로 보이는 이미지는 직접적인 인상을 담고 초록에서 파랑으로 갈수록 근본적 이미지 혹은 반전을 담는다. 인간과 자연 식물과 동물이 이들의 주 소재다. 퀸타닐라는 생물학에서 디자인으로 전공을 틀었지만 자연이 가진 복잡성을 여전히 사랑한다고 했다. 이들은 2007년 밀라노 디자인 위크에서 처음 작품을 선보였다. 빨간색 빛을 비추면 숨겨진 이미지가 나오는 작품. 이 시도가 호응을 

In [23]:
# COHERE_RE (nlist : 128, metric_type : COSINE)
result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451154865982083523,
  'distance': 0.35058271884918213,
  'entity': {'filename': 'NPRW1900000058',
   'date': 20161208,
   'NewsPaper': '매일경제신문사',
   'Topic': 'IT/과학',
   'News': '1초만에 1.75 점프진화하는 로봇 로봇은 화성에 갈 수 있다. 하지만 로봇에 뛰면서 묘기를 부려 봐라고 하면 움직이지 않는다. 기술적으로 불가능하기 때문이다. 미국 연구진이 이 같은 한계를 뛰어넘는 새로운 로봇을 개발했다. 도심 구조물을 뛰어넘으며 묘기를 부리는 스포츠인 파쿠르를 하듯 점프를 한 뒤 수직으로 된 벽을 한 번 더 차고 오르기까지 한다. 연구진은 향후 이 로봇을 재난 현장에 활용한다는 계획이다. 덩컨 홀데인 미국 버클리 기계공학과 교수 연구진은 갈라고원숭이를 모방해 파쿠르를 할 수 있는 26 크기 로봇 살토사진를 개발했다고 밝혔다. 연구 결과는 사이언스 로보틱스 7일자현지시간에 게재됐다. 연구진은 미국 연방위기관리청이 만든 수사 및 구조 훈련지를 둘러보던 중 돌무더기가 무작위로 흩어져 있는 재난 현장에서 로봇을 사용하려면 작으면서도 빈 공간을 빠르게 이동할 수 있는 로봇이 필요하다고 생각했다. 홀데인 교수는 인간을 포함해 벼룩 산양 등 20여 종 동물을 분석했다며 빠르고 높게 뛰는 동물 특성을 로봇에 적용하려 시도했다고 설명했다. 고민 끝에 연구진이 선택한 종은 갈라고원숭이였다. 아프리카 대륙에 사는 영장류인 갈라고원숭이는 파워 모듈레이션이라 불리는 독특한 특징을 갖고 있다. 갈라고원숭이는 힘줄에 에너지를 저장했다가 이를 빠르게 분출시킬 수 있는데 이때 발생하는 힘은 평소 근육이 갖고 있는 에너지의 15배에 달한다. 연구진은 이를 모방해 힘줄 대신 용수철을 이용해 갈라고원숭이처럼 힘을 축적했다가 한번에 터뜨릴 수 있는 로봇 살토를 개발했다. 무게 100 다리까지 모두 늘렸을 때 길이 26인 살토는 모터가 혼자 낼 수 

In [19]:
# COHERE_IP (nlist : 128, metric_type : IP)
result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451218565613035191,
  'distance': 0.3505191206932068,
  'entity': {'filename': 'NPRW1900000058',
   'date': 20161208,
   'NewsPaper': '매일경제신문사',
   'Topic': 'IT/과학',
   'News': '1초만에 1.75 점프진화하는 로봇 로봇은 화성에 갈 수 있다. 하지만 로봇에 뛰면서 묘기를 부려 봐라고 하면 움직이지 않는다. 기술적으로 불가능하기 때문이다. 미국 연구진이 이 같은 한계를 뛰어넘는 새로운 로봇을 개발했다. 도심 구조물을 뛰어넘으며 묘기를 부리는 스포츠인 파쿠르를 하듯 점프를 한 뒤 수직으로 된 벽을 한 번 더 차고 오르기까지 한다. 연구진은 향후 이 로봇을 재난 현장에 활용한다는 계획이다. 덩컨 홀데인 미국 버클리 기계공학과 교수 연구진은 갈라고원숭이를 모방해 파쿠르를 할 수 있는 26 크기 로봇 살토사진를 개발했다고 밝혔다. 연구 결과는 사이언스 로보틱스 7일자현지시간에 게재됐다. 연구진은 미국 연방위기관리청이 만든 수사 및 구조 훈련지를 둘러보던 중 돌무더기가 무작위로 흩어져 있는 재난 현장에서 로봇을 사용하려면 작으면서도 빈 공간을 빠르게 이동할 수 있는 로봇이 필요하다고 생각했다. 홀데인 교수는 인간을 포함해 벼룩 산양 등 20여 종 동물을 분석했다며 빠르고 높게 뛰는 동물 특성을 로봇에 적용하려 시도했다고 설명했다. 고민 끝에 연구진이 선택한 종은 갈라고원숭이였다. 아프리카 대륙에 사는 영장류인 갈라고원숭이는 파워 모듈레이션이라 불리는 독특한 특징을 갖고 있다. 갈라고원숭이는 힘줄에 에너지를 저장했다가 이를 빠르게 분출시킬 수 있는데 이때 발생하는 힘은 평소 근육이 갖고 있는 에너지의 15배에 달한다. 연구진은 이를 모방해 힘줄 대신 용수철을 이용해 갈라고원숭이처럼 힘을 축적했다가 한번에 터뜨릴 수 있는 로봇 살토를 개발했다. 무게 100 다리까지 모두 늘렸을 때 길이 26인 살토는 모터가 혼자 낼 수 있

In [22]:
# query 형식으로 들어가지 않으니 document로 넣어서 유사도 검색을 해야겠다
query_embedding = co.embed(texts=[query], input_type="search_document", model="embed-multilingual-v3.0").embeddings
query_embedding = np.asarray(query_emb)
# query_embedding.shape # (1, 1024)
query_embedding = query_embedding.reshape(-1)
# query_embedding.shape # (1024, )

result = collection.search(
    data = [query_embedding], # 앞서 생성한 동일한 크기의 임베딩 벡터 
    anns_field = EMBEDDING_FIELD_NAME, # 어떤 임베딩 필드를 기준으로 검색할 지 선언
    param = INDEX_PARAMS, # 생성했었을 때의 동일한 index parameter
    limit = 5, # top K를 뜻하며 검색해서 보고 싶은 개수
    output_fields = ['filename','date', 'NewsPaper', 'Topic', 'News'] # 보고 싶은 필드 명시
)

result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451218565613035191,
  'distance': 0.3505191206932068,
  'entity': {'date': 20161208,
   'NewsPaper': '매일경제신문사',
   'Topic': 'IT/과학',
   'News': '1초만에 1.75 점프진화하는 로봇 로봇은 화성에 갈 수 있다. 하지만 로봇에 뛰면서 묘기를 부려 봐라고 하면 움직이지 않는다. 기술적으로 불가능하기 때문이다. 미국 연구진이 이 같은 한계를 뛰어넘는 새로운 로봇을 개발했다. 도심 구조물을 뛰어넘으며 묘기를 부리는 스포츠인 파쿠르를 하듯 점프를 한 뒤 수직으로 된 벽을 한 번 더 차고 오르기까지 한다. 연구진은 향후 이 로봇을 재난 현장에 활용한다는 계획이다. 덩컨 홀데인 미국 버클리 기계공학과 교수 연구진은 갈라고원숭이를 모방해 파쿠르를 할 수 있는 26 크기 로봇 살토사진를 개발했다고 밝혔다. 연구 결과는 사이언스 로보틱스 7일자현지시간에 게재됐다. 연구진은 미국 연방위기관리청이 만든 수사 및 구조 훈련지를 둘러보던 중 돌무더기가 무작위로 흩어져 있는 재난 현장에서 로봇을 사용하려면 작으면서도 빈 공간을 빠르게 이동할 수 있는 로봇이 필요하다고 생각했다. 홀데인 교수는 인간을 포함해 벼룩 산양 등 20여 종 동물을 분석했다며 빠르고 높게 뛰는 동물 특성을 로봇에 적용하려 시도했다고 설명했다. 고민 끝에 연구진이 선택한 종은 갈라고원숭이였다. 아프리카 대륙에 사는 영장류인 갈라고원숭이는 파워 모듈레이션이라 불리는 독특한 특징을 갖고 있다. 갈라고원숭이는 힘줄에 에너지를 저장했다가 이를 빠르게 분출시킬 수 있는데 이때 발생하는 힘은 평소 근육이 갖고 있는 에너지의 15배에 달한다. 연구진은 이를 모방해 힘줄 대신 용수철을 이용해 갈라고원숭이처럼 힘을 축적했다가 한번에 터뜨릴 수 있는 로봇 살토를 개발했다. 무게 100 다리까지 모두 늘렸을 때 길이 26인 살토는 모터가 혼자 낼 수 있는 힘의 최소 2.94배나 많은 힘을 낼 수 있으며 1초 만

In [24]:
# 질문 형식으로 뽑아서 넣으면?
query = "한국 클라우드 시장의 주안점이 될 내용들이 궁금해."
query_embedding = co.embed(texts=[query], input_type="search_query", model="embed-multilingual-v3.0").embeddings
query_embedding = np.asarray(query_emb)
# query_embedding.shape # (1, 1024)
query_embedding = query_embedding.reshape(-1)
# query_embedding.shape # (1024, )

result = collection.search(
    data = [query_embedding], # 앞서 생성한 동일한 크기의 임베딩 벡터 
    anns_field = EMBEDDING_FIELD_NAME, # 어떤 임베딩 필드를 기준으로 검색할 지 선언
    param = INDEX_PARAMS, # 생성했었을 때의 동일한 index parameter
    limit = 5, # top K를 뜻하며 검색해서 보고 싶은 개수
    output_fields = ['filename','date', 'NewsPaper', 'Topic', 'News'] # 보고 싶은 필드 명시
)

result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451218565613035191,
  'distance': 0.3505191206932068,
  'entity': {'Topic': 'IT/과학',
   'News': '1초만에 1.75 점프진화하는 로봇 로봇은 화성에 갈 수 있다. 하지만 로봇에 뛰면서 묘기를 부려 봐라고 하면 움직이지 않는다. 기술적으로 불가능하기 때문이다. 미국 연구진이 이 같은 한계를 뛰어넘는 새로운 로봇을 개발했다. 도심 구조물을 뛰어넘으며 묘기를 부리는 스포츠인 파쿠르를 하듯 점프를 한 뒤 수직으로 된 벽을 한 번 더 차고 오르기까지 한다. 연구진은 향후 이 로봇을 재난 현장에 활용한다는 계획이다. 덩컨 홀데인 미국 버클리 기계공학과 교수 연구진은 갈라고원숭이를 모방해 파쿠르를 할 수 있는 26 크기 로봇 살토사진를 개발했다고 밝혔다. 연구 결과는 사이언스 로보틱스 7일자현지시간에 게재됐다. 연구진은 미국 연방위기관리청이 만든 수사 및 구조 훈련지를 둘러보던 중 돌무더기가 무작위로 흩어져 있는 재난 현장에서 로봇을 사용하려면 작으면서도 빈 공간을 빠르게 이동할 수 있는 로봇이 필요하다고 생각했다. 홀데인 교수는 인간을 포함해 벼룩 산양 등 20여 종 동물을 분석했다며 빠르고 높게 뛰는 동물 특성을 로봇에 적용하려 시도했다고 설명했다. 고민 끝에 연구진이 선택한 종은 갈라고원숭이였다. 아프리카 대륙에 사는 영장류인 갈라고원숭이는 파워 모듈레이션이라 불리는 독특한 특징을 갖고 있다. 갈라고원숭이는 힘줄에 에너지를 저장했다가 이를 빠르게 분출시킬 수 있는데 이때 발생하는 힘은 평소 근육이 갖고 있는 에너지의 15배에 달한다. 연구진은 이를 모방해 힘줄 대신 용수철을 이용해 갈라고원숭이처럼 힘을 축적했다가 한번에 터뜨릴 수 있는 로봇 살토를 개발했다. 무게 100 다리까지 모두 늘렸을 때 길이 26인 살토는 모터가 혼자 낼 수 있는 힘의 최소 2.94배나 많은 힘을 낼 수 있으며 1초 만에 1.75를 뛰어오르는 등 민첩하게 움직일 수 있다. 마치 스턴트맨처럼 공중에 떠 있